In [28]:
import tensorflow as tf

In [29]:
daily_sales = [45,668,21,69,41,-854,63,21,-656]

In [30]:
daily_sales

[45, 668, 21, 69, 41, -854, 63, 21, -656]

In [31]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales)

In [32]:
tf_dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [33]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

45
668
21
69
41
-854
63
21
-656


In [34]:
tf_dataset = tf_dataset.filter(lambda x: x > 0)

In [35]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

45
668
21
69
41
63
21


In [36]:
tf_dataset = tf_dataset.map(lambda x: x * 166)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

7470
110888
3486
11454
6806
10458
3486


In [37]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

110888
3486
6806
11454
7470
10458
3486


In [38]:
for sales in tf_dataset.batch(2).as_numpy_iterator():
    print(sales)

[  7470 110888]
[6806 3486]
[ 3486 11454]
[10458]


In [39]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales)
tf_dataset = tf_dataset.filter(lambda x: x > 0).map(lambda y: y * 166).shuffle(3).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[7470 3486]
[110888  10458]
[3486 6806]
[11454]


In [68]:
images_ds = tf.data.Dataset.list_files("Training/PlantVillage/*/*", shuffle=False)

In [69]:
for image in images_ds.take(10).as_numpy_iterator():
    print(image)

b'Training\\PlantVillage\\Early Blight\\001187a0-57ab-4329-baff-e7246a9edeb0___RS_Early.B 8178.JPG'
b'Training\\PlantVillage\\Early Blight\\002a55fb-7a3d-4a3a-aca8-ce2d5ebc6925___RS_Early.B 8170.JPG'
b'Training\\PlantVillage\\Early Blight\\009c8c31-f22d-4ffd-8f16-189c6f06c577___RS_Early.B 7885.JPG'
b'Training\\PlantVillage\\Early Blight\\00d8f10f-5038-4e0f-bb58-0b885ddc0cc5___RS_Early.B 8722.JPG'
b'Training\\PlantVillage\\Early Blight\\0182e991-97f0-4805-a1f7-6e1b4306d518___RS_Early.B 7015.JPG'
b'Training\\PlantVillage\\Early Blight\\02578b86-b234-4ac0-9bc3-691b5610e2bf___RS_Early.B 7562.JPG'
b'Training\\PlantVillage\\Early Blight\\0267d4ca-522e-4ca0-b1a2-ce925e5b54a2___RS_Early.B 7020.JPG'
b'Training\\PlantVillage\\Early Blight\\028f9b73-142f-499a-9c7b-d7c1ed5e5506___RS_Early.B 8546.JPG'
b'Training\\PlantVillage\\Early Blight\\034959c1-f1e8-4a79-a6d5-3c1d14efa2f3___RS_Early.B 7136.JPG'
b'Training\\PlantVillage\\Early Blight\\03b0d3c1-b5b0-48f4-98aa-f8904670290f___RS_Early.B 7051.JPG'


In [70]:
images_ds = images_ds.shuffle(400)
for image in images_ds.take(10).as_numpy_iterator():
    print(image)

b'Training\\PlantVillage\\Early Blight\\1d038d64-7f3c-4b4d-9b3d-7736f97b59a0___RS_Early.B 7994.JPG'
b'Training\\PlantVillage\\Early Blight\\5cb0b99b-2e14-43b3-aa51-51a7ce668793___RS_Early.B 7455.JPG'
b'Training\\PlantVillage\\Early Blight\\08392b44-ecc6-4f38-8566-361b552cfe21___RS_Early.B 7393.JPG'
b'Training\\PlantVillage\\Early Blight\\667c593c-acbf-468a-91a3-6fd8006c46c2___RS_Early.B 7830.JPG'
b'Training\\PlantVillage\\Early Blight\\34d7e068-10e6-4ea0-8547-c90937fccbbc___RS_Early.B 8198.JPG'
b'Training\\PlantVillage\\Early Blight\\12ebe6fe-c8ce-4a00-866f-8776aca75159___RS_Early.B 6873.JPG'
b'Training\\PlantVillage\\Early Blight\\0fd22207-cb37-42a1-965f-7dfe43f9d579___RS_Early.B 6876.JPG'
b'Training\\PlantVillage\\Early Blight\\3a314933-9e92-4c90-acc2-656a04c591e7___RS_Early.B 8906.JPG'
b'Training\\PlantVillage\\Early Blight\\05c35093-11b8-4cd0-b67a-148859754440___RS_Early.B 8939.JPG'
b'Training\\PlantVillage\\Early Blight\\63f59fdf-973f-4d9e-a1ac-4a21cdfb9393___RS_Early.B 8794.JPG'


In [71]:
image_count = len(images_ds)

In [72]:
train_size = int(image_count*0.8)

In [73]:
train_imgs = images_ds.take(train_size)

In [74]:
test_size = image_count - train_size

In [75]:
test_imgs = images_ds.skip(train_size)

In [76]:
classNames = ["Early Blight", "Late Blight", "Healthy"]

In [79]:
def labelFinder(filePath):
    import os
    return tf.strings.split(filePath, os.path.sep)[-2]

In [82]:
def processImg(filePath):
    img = tf.io.read_file(filePath)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    label = labelFinder(filePath)
    return img, label

In [86]:
train_imgs = train_imgs.map(processImg)
for img, label in train_imgs.take(5).as_numpy_iterator():
    print(img)
    print(label)

[[[123.75 120.75 129.75]
  [120.   117.   126.  ]
  [129.   126.   135.  ]
  ...
  [181.25 180.25 188.25]
  [174.   173.   181.  ]
  [174.25 173.25 181.25]]

 [[141.25 138.25 147.25]
  [127.75 124.75 133.75]
  [123.75 120.75 129.75]
  ...
  [180.75 179.75 187.75]
  [181.5  180.5  188.5 ]
  [177.5  176.5  184.5 ]]

 [[128.25 125.25 134.25]
  [119.   116.   125.  ]
  [121.75 118.75 127.75]
  ...
  [174.5  173.5  181.5 ]
  [175.5  174.5  182.5 ]
  [174.75 173.75 181.75]]

 ...

 [[126.5  124.5  137.5 ]
  [111.5  109.5  122.5 ]
  [112.25 110.25 123.25]
  ...
  [151.25 150.25 164.25]
  [149.   148.   162.  ]
  [147.75 146.75 160.75]]

 [[137.5  135.5  148.5 ]
  [120.5  118.5  131.5 ]
  [132.   130.   143.  ]
  ...
  [145.   147.   160.  ]
  [147.5  149.5  162.5 ]
  [149.   151.   164.  ]]

 [[132.   130.   143.  ]
  [118.25 116.25 129.25]
  [119.5  117.5  130.5 ]
  ...
  [151.25 153.25 166.25]
  [146.75 148.75 161.75]
  [152.25 154.25 167.25]]]
b'Early Blight'
[[[104.75 107.75 116.75]
  [11

In [87]:
def scale(img, label):
    return img/255, label

In [88]:
train_imgs = train_imgs.map(scale)
for img, label in train_imgs.take(5).as_numpy_iterator():
    print(img[0][0])
    print(label)

[0.42647058 0.43823528 0.50490195]
b'Early Blight'
[0.6039216  0.5921569  0.61960787]
b'Early Blight'
[0.68921566 0.6970588  0.7441176 ]
b'Early Blight'
[0.50784314 0.49215686 0.5470588 ]
b'Early Blight'
[0.5901961  0.58235294 0.6372549 ]
b'Early Blight'
